In [ ]:
!pip install mediapipe==0.10.21 opencv-python filterpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from google.colab import files
uploaded = files.upload()

# Suppose your uploaded file name = example.mp4
input_video = list(uploaded.keys())[0]
input_video


Saving How_I_Like_To_Curl_A_Barbell_Comfortable_Effective_V2.mp4 to How_I_Like_To_Curl_A_Barbell_Comfortable_Effective_V2.mp4


'How_I_Like_To_Curl_A_Barbell_Comfortable_Effective_V2.mp4'

In [6]:

import cv2
import mediapipe as mp
import numpy as np
import math
from collections import deque

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# ------------------ Text with Background ------------------ #
def draw_text_with_bg(img, text, pos, font=cv2.FONT_HERSHEY_SIMPLEX,
                      scale=0.7, color=(255,255,255), thickness=2,
                      bg_color=(0,0,0), alpha=0.6, padding=6):

    x, y = map(int, pos)
    (w, h), baseline = cv2.getTextSize(text, font, scale, thickness)

    x1 = x - padding
    y1 = y - h - padding
    x2 = x + w + padding
    y2 = y + baseline + padding

    overlay = img.copy()
    cv2.rectangle(overlay, (x1, y1), (x2, y2), bg_color, -1)
    img[:] = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

    cv2.putText(img, text, (x, y), font, scale, color, thickness, cv2.LINE_AA)


# ------------------ Angle Function ------------------ #
def angle(a, b, c):
    a = np.array(a[:2]); b = np.array(b[:2]); c = np.array(c[:2])
    ba = a - b; bc = c - b
    denom = (np.linalg.norm(ba) * np.linalg.norm(bc)) + 1e-8
    if denom == 0:
        return 0
    cos = np.dot(ba, bc) / denom
    cos = float(np.clip(cos, -1, 1))
    return float(np.degrees(np.arccos(cos)))


# ------------------ Form Evaluator ------------------ #
class FormEvaluator:
    def __init__(self, smooth_window=5):
        self.left_hist = deque(maxlen=smooth_window)
        self.right_hist = deque(maxlen=smooth_window)

    def evaluate(self, lm):
        if lm is None:
            return ["No person detected"], None, None

        # MediaPipe landmark indices
        L_SH, R_SH = 11, 12
        L_EL, R_EL = 13, 14
        L_WR, R_WR = 15, 16
        L_HP, R_HP = 23, 24

        get = lambda i: lm[i][:2]

        L_ang = angle(get(L_SH), get(L_EL), get(L_WR))
        R_ang = angle(get(R_SH), get(R_EL), get(R_WR))

        self.left_hist.append(L_ang)
        self.right_hist.append(R_ang)

        L_sm = float(np.mean(self.left_hist))
        R_sm = float(np.mean(self.right_hist))

        fb = []

        # BICEP CURL RULES
        if L_sm < 80: fb.append("Left Curl: GOOD")
        elif L_sm < 110: fb.append("Left Curl: PARTIAL")
        else: fb.append("Left Curl: INCOMPLETE")

        if R_sm < 80: fb.append("Right Curl: GOOD")
        elif R_sm < 110: fb.append("Right Curl: PARTIAL")
        else: fb.append("Right Curl: INCOMPLETE")

        # ---- SQUAT CHECK (Hip-Knee-Ankle angle) ----
        L_KN, R_KN = 25, 26  # knee points
        L_AN, R_AN = 27, 28  # ankle points

        left_squat_angle = angle(get(L_HP), get(L_KN), get(L_AN))
        right_squat_angle = angle(get(R_HP), get(R_KN), get(R_AN))

        if left_squat_angle < 95 and right_squat_angle < 95:
            fb.append("Squat Depth: Good")
        else:
            fb.append("Squat Depth: Shallow")


        # ---- PUSHUP BODY LINE CHECK ----
        body_line_angle = angle(get(L_SH), get(L_HP), get(L_AN))
        if body_line_angle < 15:
            fb.append("Push-Up Form: Straight Body")
        else:
            fb.append("Push-Up Form: Hips Sagging or Lifting")


        # TORSO SYMMETRY
        def slope(p1, p2):
            dx = p2[0] - p1[0]
            dy = p2[1] - p1[1]
            return dy / (dx + 1e-8)

        ls, rs = get(L_SH), get(R_SH)
        lh, rh = get(L_HP), get(R_HP)

        sh_slope = slope(ls, rs)
        hp_slope = slope(lh, rh)

        sh_angle = abs(math.degrees(math.atan(sh_slope)))
        hp_angle = abs(math.degrees(math.atan(hp_slope)))
        torso_diff = abs(sh_angle - hp_angle)

        if torso_diff > 12:
            fb.append(f"Torso Tilt Detected ({torso_diff:.1f}°)")
        else:
            fb.append("Torso Stable")

        # LATERAL RAISE RULE
        torso_h = abs(ls[1] - lh[1]) + 1e-6

        if abs(get(L_WR)[1] - ls[1]) < 0.25 * torso_h:
            fb.append("Left Raise: Level")
        else:
            fb.append("Left Raise: Not Level")

        if abs(get(R_WR)[1] - rs[1]) < 0.25 * torso_h:
            fb.append("Right Raise: Level")
        else:
            fb.append("Right Raise: Not Level")

        return fb, L_sm, R_sm


# ------------------ VIDEO PROCESSING (COLAB SAFE) ------------------ #
def process_video(input_path, output_path="processed.mp4"):
    pose = mp_pose.Pose(min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    w = int(cap.get(3)) or 640
    h = int(cap.get(4)) or 480

    out = cv2.VideoWriter(output_path,
                          cv2.VideoWriter_fourcc(*"mp4v"),
                          fps, (w, h))

    evaluator = FormEvaluator()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = pose.process(rgb)

        lm = None
        if res.pose_landmarks:
            lm = [[p.x*w, p.y*h, p.z, p.visibility] for p in res.pose_landmarks.landmark]

        fb, L_ang, R_ang = evaluator.evaluate(lm)

        if res.pose_landmarks:
            mp_drawing.draw_landmarks(frame, res.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        y = 30
        for line in fb:
            draw_text_with_bg(frame, line, (10, y))
            y += 35

        left_elbow_text = f"Left Elbow: {L_ang:.1f}°" if L_ang is not None else "Left Elbow: N/A"
        right_elbow_text = f"Right Elbow: {R_ang:.1f}°" if R_ang is not None else "Right Elbow: N/A"

        draw_text_with_bg(frame, left_elbow_text, (10, h-60))
        draw_text_with_bg(frame, right_elbow_text, (10, h-25))

        out.write(frame)

    cap.release()
    out.release()

    return output_path


In [7]:
output = process_video(input_video, "processed.mp4")
output


'processed.mp4'

In [8]:
files.download("processed.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>